In [1]:
import subprocess as sp
import numpy as np
import pandas as pd
from io import StringIO
import os
import re
import shutil

from utils import *

In [2]:
#global parameters
cudadir = "/usr/common/software/cuda/10.0.130"

In [6]:
#input and output dirs
datadirs = ["./data/tf_2.0b/good"] #,"./data/tf_2.0b/investigate"]
outputdir = "./data/tf_2.0b/sanitized"

# Functions

In [10]:
def compare_sets(set1, set2):
    good = True
    not_in_set1 = set()
    not_in_set2 = set()
    for item in set1:
        if item in set2:
            set2.remove(item)
        else:
            #print(item, "in set1 but not in set2")
            not_in_set2.add(item)
            good = False
    
    for item in set2:
        #print(item,"in set2 but not in set1")
        not_in_set1.add(item)
        good = False
        
    return good, not_in_set1, not_in_set2

# Check for Missing Data

In [11]:
#get metric list
files = []
for datadir in datadirs:
    files += [ os.path.join(datadir,x) for x in os.listdir(datadir) if (os.path.splitext(x)[-1] == ".nvprof") or (os.path.splitext(x)[-1] == ".nvvp") ]

#recs
records = []

#build feature list:
for path in files:
    
    #filename
    file = os.path.basename(path)
    
    #path
    path = os.path.dirname(path)
    
    #splitup
    splt = file.split(".")
    
    prefix = ".".join(splt[0:-2])
    metric = splt[-2].split("metric_")[1]
    
    #append to records
    records.append({"prefix": prefix, "metric": metric, "file": os.path.join(path, file)})

#put in df
recorddf = pd.DataFrame(records).sort_values(["prefix", "metric"])

#get all metrics
all_metrics = list(recorddf["metric"].unique())

#group by metric:
missingrecorddf = pd.DataFrame(recorddf.groupby("prefix").apply(lambda x: pd.Series([y for y in all_metrics if y not in list(x["metric"])])))

#create exclusion list:
excludelist = list(missingrecorddf.reset_index()["prefix"].unique())

#print the missing ones
missingrecorddf

KeyError: 'prefix'

# Read Kernels in each File

In [9]:
#sort by those keys:
sortkeys = ["Network Name", "Input Shape", "Kernel Shape", \
            "Batch Size", "Stride Size", "Data Format", "Pass", \
            "Precision", "Device", "Name", "Metric Name"]

#limit the input
#recorddf = recorddf[ recorddf["prefix"].str.startswith("profile.name_ResNet50-2.batchsize_16.inputshape_112x112x64.kernelshape_7x7x64x64.stride_2.dataformat_NHWC.fp32") ]

#group by prefixes and files
all_prefixes = set([x.split(".pass")[0] for x in recorddf["prefix"]])
all_metrics = recorddf["metric"].unique()
all_passes = set([x.split(".pass_")[1].replace(".pass_","") for x in recorddf["prefix"].unique()])

#metrics
df_profiles = []

for pref in all_prefixes:
    
    #loop over passes
    for pas in all_passes:
        
        #project frame
        selectdf = recorddf[ recorddf["prefix"] == pref + ".pass_" + pas ]
        
        if selectdf.empty:
            continue
        
        #loop over metrics
        df_summary = []
        for met in [x for x in all_metrics if x != "time"]:
            
            #filename
            file = selectdf.loc[ selectdf["metric"] == met, "file" ].values[0]
        
            #extract metric name
            parameters, metric = parse_filename(os.path.basename(file))
            metrics = metric.split("-")
        
            #import as summary
            tmpdf = import_nvprof_metric(file, timeline=False, cuda_dir=cudadir).sort_values(by="Name").reset_index(drop=True)
            for key in parameters:
                tmpdf[key] = parameters[key]
            del tmpdf["Metric Description"]
            tmpdf["Metric Name"] = met
            
            #append to summary
            df_summary.append(tmpdf)
        
        #concat
        metricdf = pd.concat(df_summary)
        metricdf = metricdf[ ["Name", "Metric Name"] ]
        
        #do time now
        file = selectdf.loc[ selectdf["metric"] == "time", "file" ].values[0]
        timedf, markerdf = import_nvprof_overview(file, cuda_dir=cudadir)
    
        #extract metric name
        parameters, _ = parse_filename(os.path.basename(file))
        for key in parameters:
            timedf[key] = parameters[key]
        
        #now, throw everything away but the kernel names and metric
        #also delete potential memcpy entry from the overview
        timedf = timedf.loc[ (timedf["Collection Type"]=="gpu_activities") & ~(timedf["Name"].str.contains("CUDA memcpy") | timedf["Name"].str.contains("CUDA memset")), ["Name"] ]
        
        #kernels
        kernels = set(timedf["Name"].unique())
        
        #now, for every metric, see if we have the same kernels as in the time df
        good = True
        for metric in metricdf["Metric Name"].unique():
            projdf = metricdf.loc[ metricdf["Metric Name"]==metric, ["Name"] ]
            tmpkernels = set(projdf["Name"].unique())
            
            tmpgood, not_in_set1, not_in_set2 = compare_sets(kernels, tmpkernels)
            good &= tmpgood
            
            if not tmpgood:
                print("discrepancy found:\n\tpass: {}\n \tmetric: {}\n \tprefix: {}\n".format(pas,metric,pref))
                print("kernels in time file but not in metric file:")
                for item in not_in_set2:
                    print("\t",item)
                print("\n")
                print("kernels in metric file but not in time file:")
                for item in not_in_set1:
                    print("\t",item)
                print("\n\n")
        #print(metricdf, timedf)
        
        #if all is good, move the files over to sanitized
        if good:
            for infile in selectdf["file"].unique():
                outfile = os.path.join(outputdir,os.path.basename(infile))
                #print(infile,outfile)
                shutil.move(infile,outfile)

discrepancy found:
	pass: forward
 	metric: atomic_transactions
 	prefix: profile.name_ResNet50-2.batchsize_16.inputshape_112x112x64.kernelshape_7x7x64x64.stride_2.dataformat_NHWC.fp32

kernels in time file but not in metric file:
	 void fft2d_r2c_32x32<float, bool=0, unsigned int=1, bool=1>(float2*, float const *, int, int, int, int, int, int, int, int, int, cudnn::reduced_divisor, bool, int2, int, int)
	 void fft2d_r2c_32x32<float, bool=0, unsigned int=0, bool=0>(float2*, float const *, int, int, int, int, int, int, int, int, int, cudnn::reduced_divisor, bool, int2, int, int)
	 volta_gcgemm_32x32_nt
	 void fft2d_c2r_32x32<float, bool=1, bool=0, unsigned int=0, bool=0, bool=0>(float*, float2 const *, int, int, int, int, int, int, int, int, int, float, float, cudnn::reduced_divisor, bool, float*, float*, int2, int, int)


kernels in metric file but not in time file:
	 volta_scudnn_128x64_relu_medium_nn_v1
	 cudnn::gemm::computeOffsetsKernel(cudnn::gemm::ComputeOffsetsParams)



discrep

In [106]:
recorddf

,file,metric,prefix
10,./data/tf_2.0b/investigate/profile.name_ResNet...,atomic_transactions,profile.name_ResNet50-2.batchsize_16.inputshap...
19,./data/tf_2.0b/investigate/profile.name_ResNet...,dram_read_transactions,profile.name_ResNet50-2.batchsize_16.inputshap...
23,./data/tf_2.0b/investigate/profile.name_ResNet...,dram_write_transactions,profile.name_ResNet50-2.batchsize_16.inputshap...
13,./data/tf_2.0b/investigate/profile.name_ResNet...,flop_count_hp,profile.name_ResNet50-2.batchsize_16.inputshap...
1,./data/tf_2.0b/investigate/profile.name_ResNet...,flop_count_sp,profile.name_ResNet50-2.batchsize_16.inputshap...
15,./data/tf_2.0b/investigate/profile.name_ResNet...,gld_transactions,profile.name_ResNet50-2.batchsize_16.inputshap...
7,./data/tf_2.0b/investigate/profile.name_ResNet...,gst_transactions,profile.name_ResNet50-2.batchsize_16.inputshap...
18,./data/tf_2.0b/investigate/profile.name_ResNet...,l2_read_transactions,profile.name_ResNet50-2.batchsize_16.inputshap...
26,./data/tf_2.0b/investigate/profile.name_ResNet...,l2_write_transactions,profile.name_ResNet50-2.batchsize_16.inputshap...
20,./data/tf_2.0b/investigate/profile.name_ResNet...,shared_load_transactions,profile.name_ResNet50-2.batchsize_16.inputshap...
